In [ ]:
import requests
import json
import pandas as pd
import time
import timeit
import math
from time import sleep

# danh sách tọa độ

In [ ]:
!pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises", timeout = 1000)
from time import sleep
r_check = 0.0185 #init 0.0184
r_delta = 0.005 #init0.0175

In [ ]:
def location(latitude_location, longitude_location):
    return str(geolocator.reverse(str(latitude_location)+","+str(longitude_location)))

def in_(latitude, longitude, name,r = r_check): # ?in district
    geolocator = Nominatim(user_agent="ungdungptdltm@gmail.com", timeout = 1000)
    lat_lons = [[latitude, longitude],[latitude + r, longitude],[latitude - r, longitude],
          [latitude, longitude - r], [latitude, longitude + r]]  # tâm trên dưới trái phải
    for lat_lon in lat_lons:
        location_ = location(lat_lon[0],lat_lon[1])
        if name in location_:
            return name
     
    geolocator = Nominatim(user_agent="Ntman262@gmail.com",timeout=1000)
    r2 = r/2
    lat_lons2 = [[latitude + r2, longitude],[latitude - r2, longitude],
          [latitude, longitude - r2], [latitude, longitude + r2]]  # tâm trên dưới trái phải
    for lat_lon in lat_lons2:
        location_ = location(lat_lon[0],lat_lon[1])
        if name in location_:
            return name
        
    geolocator = Nominatim(user_agent="geoapiExercises", timeout = 1000)    
    r3 = r_delta
    lat_lons3 = [[latitude + r3, longitude],[latitude - r3, longitude],
          [latitude, longitude - r3], [latitude, longitude + r3]]  # tâm trên dưới trái phải
    for lat_lon in lat_lons3:
        location_ = location(lat_lon[0],lat_lon[1])
        if name in location_:
            return name
    return 0

def get_locations(district,delta=r_delta):
    name = district[0]
    lat_0 = district[1]; m_lat=district[2] 
    lon_0 = district[3]; m_lon=district[4]

    locations = []
    lat = lat_0
    while 1:
        lon = lon_0
        while 1:
            if in_(lat, lon,name ):
                locations.append([lat, lon,name])
            if lon - delta >m_lon:
                break
            lon = lon + 2*delta
        if lat - delta > m_lat:
            break
        lat = lat + 2*delta
        sleep(2)

    lat = lat_0 + delta
    while 1: #10.375976 - 11.159858, 106.363978-107.026852
        lon = lon_0 + delta
        while 1:
            if in_(lat, lon,name ):
                locations.append([lat, lon,name])
            if lon - delta >m_lon:
                break
            lon = lon + 2*delta
        if lat - delta > m_lat:
            break
        lat = lat + 2*delta
        sleep(2)
    return locations

### địa bàn TPHCM

In [ ]:
# # 10.375976 - 11.159858, 106.363978-107.026852
# %%time
# geolocator = Nominatim(user_agent="geoapiExercises",timeout = 1000)
HCMC = ["Thành phố Hồ Chí Minh", 10.375976 , 11.159858, 106.363978,107.026852]
all_HCMC = get_locations(HCMC)
all_HCMC_df = pd.DataFrame (all_HCMC, columns = ['lat', 'lon', 'name_district'])
all_HCMC_df.to_excel("all_HCMC.xlsx")

locations_all_df = all_HCMC_df
len(all_HCMC_df)

## Browsing ID

In [ ]:
get_browsing_id_header={
    'accept': 'application/json, text/plain, */*'
,'x-foody-client-id': 'fa42efc4-7303-4cd8-b71f-93359fb6d50a' # thay đổi theo user của bạn
,'x-foody-client-type': '4'
,'x-foody-app-type': '1004'
,'x-foody-client-version': '5.23.0'
,'x-foody-api-version': '1'
,'x-foody-client-language': 'en'
# ,'x-foody-access-token': #unused
,'user-agent': 'NOW/5.13.2 (sdk_gphone64_x86_64; android 33; Scale/2.75)'
,'x-foody-client-rn-version': '5.13.2'
# ,'x-foody-device-fingerprint': #unused 
,'content-type': 'application/json; charset=UTF-8'
,'content-length': '110'
,'accept-encoding': 'gzip'
}

get_browsing_id_payload={
    "city_id": 217,
    "foody_services": [
        1
    ],
    "position": {
        "latitude": 10.837747, #toạ độ tâm bán kính 2km 
        "longitude": 106.7211258 #
    },
    "sort_type": 30
}

In [ ]:
def get_browsing_ids(url,header_param, payload_param):
    header=header_param.copy()
    payload=payload_param.copy()
    header['content-length']=str(len(json.dumps(payload, separators=(',',':'))))
    respone=requests.post(url,headers=header,json=payload)
    data=json.loads(respone.text)
    return data['data']['delivery_ids']

In [ ]:
def get_ids(locations):
    len_ = []
    browsing_ids = []
    url = 'https://gappapi.deliverynow.vn/api/v5/buyer/delivery/get_browsing_ids'
    for i in range(len(locations)):
        location_1=round(locations.lat[i],7),round(locations.lon[i],7)
        get_browsing_id_payload['position']['latitude']=location_1[0]
        get_browsing_id_payload['position']['longitude']=location_1[1]
        get_browsing_id_payload
        browsing_ids_=get_browsing_ids(url,get_browsing_id_header,get_browsing_id_payload)   
        len_.append(len(browsing_ids_))
        browsing_ids = browsing_ids + browsing_ids_
    return browsing_ids, len_

In [ ]:
list_ids, list_len = get_ids(locations_all_df)

In [ ]:
locations_all_df["len"] = list_len
locations_all_df.to_excel("locations.xlsx")

In [ ]:
ids_df = pd.DataFrame(list_ids, columns=['id'])
len(ids_df)

In [ ]:
df1=ids_df.drop_duplicates()
len(df1)

In [ ]:
df2 = df1.reset_index()

In [ ]:
cols = ['id','lat','lon']
def fix(i):
    l1 = 0
    for j in range(len(list_len)):
        len2 = list_len[j]
        l1 = l1 + len2
        if i<l1:
            v= [df1.id[i],locations_all_df.lat[j] , locations_all_df.lon[j]]
            return dict(zip(cols, v))

out =df2['index'].apply(fix)
out_df = pd.DataFrame(list(out))

In [ ]:
out_df

In [ ]:
out_df.to_excel("ids.xlsx",index=None)